In [7]:
import numpy as np
import pandas as pd
import random
from typing import List
from itertools import combinations
from geopy.distance import geodesic

## Inizialization

In [32]:
POPULATION_SIZE = 100              # Dimensione della popolazione
OFFSPRING_SIZE = 50                # Numero di discendenti per generazione
MAX_GENERATIONS = 1000             # Numero massimo di generazioni
MUTATION_RATE = 0.3                # Probabilità di mutazione

# Caricamento delle città e costruzione della matrice delle distanze
CITIES = pd.read_csv('./italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

# Parametri per l'algoritmo genetico
NUM_CITIES = len(CITIES)           # Numero di città


def initialize_population(size: int, start_city: str ) -> List[List[int]]:
    # Trova l'indice della città di partenza
    start_city_index = CITIES[CITIES['name'] == start_city].index[0]
    
    # Lista degli indici delle città escludendo quella di partenza
    other_cities = [i for i in range(NUM_CITIES) if i != start_city_index]
    
    # Genera la popolazione fissando la città di partenza e arrivo
    population = []
    for _ in range(size):
        individual = [start_city_index]  # Messina come primo elemento
        individual += list(np.random.permutation(other_cities))  # Permutazione delle altre città
        individual.append(start_city_index)  # Messina come ultimo elemento
        population.append(individual)
    
    return population


## Fitness

In [34]:
# Funzione per calcolare il costo del percorso (lunghezza totale)
def path_cost(path: List[int]) -> float:
    return sum(DIST_MATRIX[path[i], path[(i + 1) % NUM_CITIES]] for i in range(NUM_CITIES))


# Funzione di fitness: inverso della lunghezza del percorso
def fitness(individual: List[int]) -> float:
    return  -path_cost(individual)

## Parent selection function

In [35]:
# Selezione dei genitori tramite torneo
def parent_selection(population: List[List[int]]) -> List[int]:
    tournament = random.sample(population, 5)
    tournament.sort(key=fitness, reverse=True)
    return tournament[0]

## Crossover functions

In [37]:
def inver_over_crossover(parent1: List[int], parent2: List[int]) -> List[int]:
    # Copia il percorso del primo genitore per iniziare a creare il figlio
    child = parent1[:]
    
    # Imposta la città di partenza e arrivo fissa (ad esempio Messina)
    start_city = child[0]
    
    # Seleziona una città casuale (gene) all'interno del percorso, esclusa la partenza e arrivo
    gene_index = random.randint(1, len(parent1) - 2)
    gene = child[gene_index]
    
  # Trova la posizione della città successiva nel secondo genitore
    next_index = (parent2.index(gene) + 1) % (len(parent2) - 1)  # Escludendo l'ultima città
    next_city = parent2[next_index]
    # Assicuriamoci che next_city non sia la città di arrivo e sia presente nel figlio
    if next_city == child[-1] or next_city not in child:
        # Se next_city è la città di arrivo, trova un'altra città da utilizzare
        for city in parent2:
            if city != start_city and city not in child:
                next_city = city
                break
    
 # Se il gene selezionato nel primo genitore non è collegato al prossimo nel secondo genitore, inverte
    if child[gene_index + 1] != next_city:
        # Trova l'indice della città successiva nel primo genitore
        next_index = child.index(next_city)

        # Esegue l'inversione della sottosequenza tra gene_index e next_index
        if gene_index < next_index:
            child[gene_index:next_index + 1] = reversed(child[gene_index:next_index + 1])
        else:
            child[next_index:gene_index + 1] = reversed(child[next_index:gene_index + 1])
    
    # Assicura che la città di partenza e di arrivo restino invariate
    child[0] = start_city
    child[-1] = start_city
    
    return child


# Operatore di crossover: Ordinal Crossover (OX)
def crossover(parent1: List[int], parent2: List[int]) -> List[int]:
    # Imposta il primo e l'ultimo elemento fisso (città di partenza/arrivo)
    start, end = sorted(random.sample(range(1, NUM_CITIES - 1), 2))
    child = [parent1[0]] + [None] * (NUM_CITIES - 2) + [parent1[0]]
    
    # Copia il segmento selezionato dal primo genitore
    child[start:end] = parent1[start:end]
    
    # Completa il percorso con città dal secondo genitore, evitando duplicati
    position = end
    for city in parent2:
        if city not in child:
            if position >= NUM_CITIES - 1:
                position = 1  # Riparte dal secondo elemento per evitare il primo e l'ultimo
            child[position] = city
            position += 1

    return child



## Mutation functions

In [40]:
# Operatore di mutazione: Scambia due città casualmente
def mutate(individual: List[int]) -> List[int]:
    if random.random() < MUTATION_RATE:
        # Seleziona due città da scambiare, escludendo la città di partenza e arrivo
        i, j = random.sample(range(1, NUM_CITIES - 1), 2)
        individual[i], individual[j] = individual[j], individual[i]
    return individual


def inversion_mutation(individual: List[int]) -> List[int]:
    # Assicura che la città di partenza e arrivo sia fissa
    if random.random() < MUTATION_RATE:
        start_city = individual[0]
        end_city = individual[-1]
        
        # Seleziona due indici casuali tra le città interne (1 a len(individual) - 2)
        idx1, idx2 = sorted(random.sample(range(1, len(individual) - 1), 2))
        
        # Inverte la sequenza tra idx1 e idx2
        individual[idx1:idx2 + 1] = reversed(individual[idx1:idx2 + 1])
        
        # Ripristina la città di partenza e arrivo
        individual[0] = start_city
        individual[-1] = end_city
    
    return individual


In [41]:

# Algoritmo genetico principale
population = initialize_population(POPULATION_SIZE, start_city="Syracuse")
for generation in range(MAX_GENERATIONS):
    # Creazione della prole
    offspring = []
    for _ in range(OFFSPRING_SIZE):
        parent1 = parent_selection(population)
        parent2 = parent_selection(population)
        child = inver_over_crossover(parent1, parent2)
        child = inversion_mutation(child)
        offspring.append(child)
    
    # Valutazione fitness della prole
    population.extend(offspring)
    population.sort(key=fitness, reverse=True)
    population = population[:POPULATION_SIZE]  # Mantiene i migliori

    # Stampa ogni 100 generazioni
    if generation % 100 == 0:
        print(f"Generation {generation}, Best Path Cost: {path_cost(population[0]):.2f} km")

# Risultato finale
best_individual = population[0]
print("Miglior percorso trovato:", [CITIES['name'][i] for i in best_individual])
print("Costo del miglior percorso:", path_cost(best_individual), "km")



Generation 0, Best Path Cost: 15011.94 km
Generation 100, Best Path Cost: 5880.64 km
Generation 200, Best Path Cost: 4641.12 km
Generation 300, Best Path Cost: 4309.96 km
Generation 400, Best Path Cost: 4267.50 km
Generation 500, Best Path Cost: 4175.99 km
Generation 600, Best Path Cost: 4175.99 km
Generation 700, Best Path Cost: 4175.99 km
Generation 800, Best Path Cost: 4175.99 km
Generation 900, Best Path Cost: 4175.99 km
Miglior percorso trovato: ['Syracuse', 'Reggio di Calabria', 'Messina', 'Taranto', 'Bari', 'Andria', 'Foggia', 'Salerno', 'Naples', 'Giugliano in Campania', 'Pescara', 'Latina', 'Rome', 'Terni', 'Perugia', 'Ancona', 'Rimini', 'Forlì', 'Ravenna', 'Trieste', 'Venice', 'Padua', 'Vicenza', 'Bolzano', 'Trento', 'Verona', 'Brescia', 'Bergamo', 'Monza', 'Milan', 'Novara', 'Turin', 'Genoa', 'Piacenza', 'Parma', "Reggio nell'Emilia", 'Modena', 'Ferrara', 'Bologna', 'Prato', 'Florence', 'Leghorn', 'Sassari', 'Cagliari', 'Palermo', 'Catania', 'Syracuse']
Costo del miglior per